In [18]:
# Colabi jaoks Google Drive'i liikumine
from google.colab import drive
# Haagime külge GD kausta - tuleb omalt poolt anda Colabile juurdepääsuluba
drive.mount('/content/drive')
# Failidega tegutsemiseks liigume GD kausta Colab Notebooks (või mujale, muutke endale sobivaks)
import os
os.chdir('/content/drive/My Drive/Bakatöö/')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!pip install estnltk==1.6.9.1b0
!pip install openpyxl==3.0.9

In [20]:
import requests
import json
import html
import time 
import re

from estnltk import Text
import estnltk
from estnltk.taggers import SpellCheckRetagger

import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer

from xlwt import Workbook
import openpyxl

In [21]:
# Meetod, et Ekilexi päringuid teha
def getData(params, findMeaning):
    if findMeaning:
        url = "https://ekilex.eki.ee/api/meaning/" + params
    else:
        url = "https://ekilex.eki.ee//api/word/search/" + params
    headers = {
        'ekilex-api-key': "542e81f143d84ddda95a93c04f23377c",
    }
    try:
        return requests.request("GET", url, headers=headers).json()
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

# Meetod, mis tagastab Ekilexis sõnaseletuse ID
def getMeaningID(userInput):
    response = getData("search/" + userInput, True)
    # Kontrollin, kas sellele sõnale leidub ikka tulemus, kui ei leidu panen vastuseks none ja tagastan False.
    meaningIDs = []
    wordSynonums = []
    if response['meaningCount']:
        for i in range(response['meaningCount']):
            wordSynonums.append(
                getSynonyms(response['results'][i], userInput)) 
            results = response['results'][i] if (response['resultCount'] != 0) else None
            meaningIDs.append(results['meaningId'])
    if not meaningIDs:
        return 'puudub', [[]]
    return meaningIDs, wordSynonums


# Meetod, mis saab kätte Ekilexis olevad sünonüümid
def getSynonyms(results, userInput):
    meaningWords = results['meaningWords']
    return [word['wordValue'].strip() for word in meaningWords if
            word['lang'] == 'est' and word['wordValue'].strip() != userInput]


# Meetod, mis tagastab seletuse ID põhjal vastava sõna seletuse.
def getWordMeaning(meaningID):
    response = getData("details/{}".format(meaningID), True)
    if len(response['definitionLangGroups']) == 0:
        return None
    definitionLangGroups = response['definitionLangGroups'][0]
    definitions = definitionLangGroups['definitions'][0]
    return html.unescape(definitions['value'])

In [22]:
# Meetod, mis teeb päringu TartuNLP Neurõtõlkesse
# Tagastab tõlkest saadud tulemuse
def getTranslation(text, src, tgt):
    url = 'https://api.tartunlp.ai/translation/v2'
    obj = {'text': text, 'src': src, 'tgt': tgt}
    x = requests.post(url, json=obj).json()
    return x['result']

def engExplanation(externals):
    for engExpl in externals:
        if engExpl['type_ref_code'] == 'eq_synonym' and engExpl['definition'] != '':
            translation = getTranslation(engExpl['definition'], 'en', 'et')
            expl = 'Neurotõlge: ' + translation.lower()
            return translation, True
    return 'puudub', False


In [23]:
# Meetod, mis genereerib seletusi mine-liitelistele sünohulkadele
def getMineForm(word):
  expl = 'puudub'
  tekst = estnltk.Text(word)
  tekst.tag_layer()
    ## 1. Iga -mine lõpuga sõna puhul peab kontrollima, kas ending on '0', text on võrdne lemmaga, form on võrdne 'sg n'. (Teen kindlaks, et tegemist on -mine vormiga)
  if tekst.text == tekst.lemma[0][0] and tekst.ending[0][0] == '0' and tekst.form[0][0] == 'sg n':
      ## 2. Võtan -mine lõpu ära ning liidan -ma lõpu. 
    algvorm = word[:-4] + 'ma'
    ids, syns = getMeaningID(algvorm)
    if ids != 'puudub':
      for id in ids: 
        explEki = getWordMeaning(id)
        if explEki is not None: break 
      if not explEki:
        return expl
      else: 
        print('Sõna millega tehti päring: ', algvorm)
        specialChars = list('[@_!#$%^&*()<>?/\|}{~:],')
        explTokenized = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(explEki)] # Kasutan NLTK-d, sest see tegi seletuse lausemärkide pealt täpsemalt pooleks, kui EstNLTK tokenize.
        ## 3. Käin läbi seletuse ning kõik -ma vormid muudan tagasi -mine vormidesse.
        for i, explWord in enumerate(explTokenized[0]):
          if explWord not in specialChars:
            tekstExpl = estnltk.Text(explWord).tag_layer()
            # Kui tegusõna on ma-vormis teisendan mine-vormi
            if (tekstExpl.ending[0][0] == 'ma' or tekstExpl.ending[0][0] == 'tama')and tekstExpl.partofspeech[0][0] == 'V':
              explTokenized[0][i] = explWord[:-2] + 'mine'
              if i == 0 and len(explTokenized[0]) >= 3:
                nextWord = explTokenized[0][i + 1] if explTokenized[0][i + 1] not in specialChars else explTokenized[0][i + 2]
                if nextWord[:-2] != 'ma' and explTokenized[0][1] not in specialChars: 
                  explTokenized[0][0], explTokenized[0][1] = explTokenized[0][1], explTokenized[0][0]
                  tekstExpl = estnltk.Text(explTokenized[0][0]).tag_layer()
            # Ainsuse osastavas või sisseütlevas käändes sõnad teisendan ainsuse omastavasse 
            if (tekstExpl.form[0][0] == 'sg p' or tekstExpl.form[0][0] == 'adt') and explWord[-4:] != 'mine':
              synthesized = estnltk.vabamorf.morf.synthesize(tekstExpl.words.lemma[0][0], 'sg g') 
              if synthesized: 
                explTokenized[0][i] = synthesized[0] if synthesized[0] != 'kee' else synthesized[1]
            if tekstExpl.form[0][0] == 'sg g' and tekstExpl.partofspeech[0][0] == 'S' and explWord[-2:] == 'ma': 
              explTokenized[0][i] = explWord[:-2] + 'mine'
            # Mitmuse osastav teisendan mitmuse omastavasse käändesse
            if tekstExpl.form[0][0] == 'pl p' and tekstExpl.partofspeech[0][0] == 'S':
              synthesized = estnltk.vabamorf.morf.synthesize(tekstExpl.words.lemma[0][0], 'pl g')
              if synthesized:
                explTokenized[0][i] = synthesized[0]
          explEki = TreebankWordDetokenizer().detokenize(explTokenized[0])
          translatorExpl =  getTranslation(explEki, 'et', 'et').lower() # Saan TartuNLP Neurotõlke (est->est) tulemuse
          expl = '{} \nNeurotõlge (est -> est): {}'.format(explEki, formatTranslatorExpl(translatorExpl))
  return expl

# Meetod selleks, et asendada neurotõlkes hr. sõnaga hrl, sest see oli pidev viga.
def formatTranslatorExpl(translatorExpl):
  expl = translatorExpl.split(' ')
  if 'hr.' in expl:
    expl[expl.index('hr.')] = 'hrl'
  elif 'härra' in expl:
    expl[expl.index('härra')] = 'hrl'
  return ' '.join(expl)
 

In [24]:
# Meetod, mis genereerib seletusi ja-liitelistele sünohulkadele
def getJaForm(word):
  expl = 'puudub'
  tekst = estnltk.Text(word)
  tekst.tag_layer()
  if tekst.text == tekst.lemma[0][0] and tekst.ending[0][0] == '0' and word[-2:] == 'ja':
    verb = word[:-2] + 'ma' # Teisendan sõna algvormi
    ids, syns = getMeaningID(verb) # Saan Ekilexist, vastava sõna seletuste ID-d
    if ids != 'puudub':
      # Käin seletuste ID-d läbi ning võtan esimese seletuse, mis pole tühi
      for id in ids: 
        explEki = getWordMeaning(id) 
        if explEki is not None: break
      if not explEki:
        return expl
      else:
        explEki = cleanEkiExpl(explEki)
        explTokenized = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(explEki)] # Saan seletusest järjendi
        # Muudan seletuse õigesse vormi.
        # Kuna tegemist on tegija-nimedega, siis lisan 'keegi, kes' seletuse ette, sest suure tõenäosusega on tegija elusolend.
        expl = 'keegi, kes ' + TreebankWordDetokenizer().detokenize(changeVerbs(explTokenized)) 
  return expl

# Korrigeerin Ekilexist saadud seletust
def cleanEkiExpl(explEki):
  if explEki[-1] == '.':
    explEki.pop()
  if explEki[0] == '(' and explEki[-1] != ')':
    i = explEki.index(')')
    explEki = explEki[i + 1:]
  if explEki[0] == '(' and explEki[-1] == ')':
    explEki = explEki[1:-1]
  return explEki

# Meetod, et muuta seletuses olevad sõnad õigesse vormi
def changeVerbs(explTokenized):
  specialChars = list('[@_!#$%^&*()<>?/\|}{~:],')
  for i, explWord in enumerate(explTokenized[0]):
    if explWord not in specialChars:
      tekstExpl = estnltk.Text(explWord).tag_layer()
      # Teisendan ma-liitega ja tama-liitega sõnad ainsuse aktiivi.
      if (tekstExpl.ending[0][0] == 'ma' or tekstExpl.ending[0][0] == 'tama')and tekstExpl.partofspeech[0][0] == 'V':
        if i != len(explTokenized[0]) - 1:
          next_word = explTokenized[0][i + 1]
          if next_word[-2:] == 'ma':
            continue
        synthesized = estnltk.vabamorf.morf.synthesize(tekstExpl.words.lemma[0][0], 'b')
        explTokenized[0][i] = synthesized[0]
  return explTokenized[0]

In [25]:
# Meetod, et saada EstWN-st kätte sünohulgad
def getMissingMeanigns(lexid, offset):
    url = "https://estwn.keeleressursid.ee//api/v1/synset/?lexid={}&offset={}".format(lexid, offset)
    try:
        resp = requests.request("GET", url)
        resp.raise_for_status()
        if resp.status_code != 204:
            resp = resp.json()
            return resp['results']
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

# Meetod, et kätte saada 
def getIDsEki(word):  # Sisse tuleb anda synset
    IDs, syns = getMeaningID(word)
    if not IDs:
        return 'puudub', [[]]
    else:
        return IDs, syns

# Meetod, et saada kätte mingi sõna esinemine erinevates EstWN-i sünohulkades
def getWord(lexid, word):
    url = "https://estwn.keeleressursid.ee//api/v1/synset/?lexid={}&word={}&mode=slim".format(lexid, word)
    try:
        resp = requests.request("GET", url)
        resp.raise_for_status()
        if resp.status_code != 204:
            resp = resp.json()
            return resp['results']
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

In [26]:
# Meetod, mis kasutab vastavalt, kas mine-tegevusnimede seletuste genereerimis meetodid või
# ja-tegijanimede seletuste genereerimismeetodit
def correctForm(unique, variants, expl, form):
  for k, v in unique.items():
    check = all(item in variants for item in list(k))
    if v == '' and check:
      for word in k:
        explEki = getMineForm(word) if form =='mine' else getJaForm(word)
        if explEki != 'puudub'and not synonymInExplanation(variants, explEki):
          print('Sõnaga, millega tehti päring: ', word)
          expl = explEki
          break
  return expl


# Kontroll, kas vähemalt 50% sünohulga liikmetest on vastavalt
# mine-tegevusenimed või ja-tegijanimed
def checkForm(variants, form, nr):
  counter = 0
  for word in variants:
    if word[-nr:] == form:
      counter += 1
  if counter / len(variants) >= 0.5:
    return True
  else:
    return False

In [27]:
# Kontroll, kas vastav seletus juba eksisteerib mingil sünohulga liikmel EstWN-is.
def explanationExists(lexid, expl, variants):
    for word in variants:
        #print(word)
        time.sleep(5)
        responses = getWord(lexid, word)
        for resp in responses:
            primaryDef = resp['primary_definition'].split(' ')
            primaryDef = ' '.join(primaryDef) if primaryDef[-1] != '(EKSS)' else ' '.join(primaryDef[:-1])
            if primaryDef == expl:
                return True
    return False


# Kontroll, kui seletus on lühem kui 3 sõna, siis ei või seal esineda ühtegi sünohulga liiget.
# Kui seletus on pikem kui 3 sõna siis võib, sest suure tõenäosusega saab seal seletuses käsitsi vastava sõna asendada
def synonymInExplanation(synonyms, expl):
    if len(expl.split(' ')) < 3:
        check = any(item in synonyms for item in expl.split(' '))
        print('Sünonuumid: {}. Seletus: {}. Tulemus: {}'.format(synonyms, expl.split(' '), check))
        return any(item in synonyms for item in expl.split(' '))
    else:
        return False


# Meetod, et vormindada sünohulk.
def processSynset(result):
    processed = []
    if result['variants_str'] != '[]':
        synsets = result['variants_str'].translate({ord(i): None for i in '[]'})
        synsets = synsets.split(',')
        for k in synsets:
            var = k.split('_')
            processed.append(var[0].strip())
    return processed


In [28]:
# Unikaalse sünohulga liikme põhjal seletuse leidmine
def uniqueExpl(expl, unique, variants, meetod):
    for key in unique:
        check = all(item in variants for item in list(key))
        if unique[key] == '' and check:
            for word in key:
                explID, syonyms = getMeaningID(word)
                if explID != 'puudub':
                    explEki = getWordMeaning(explID[0])
                    if explEki != 'puudub' and explEki is not None:
                      if len(explEki.split(' ')) > 1 and \
                            not synonymInExplanation(variants, explEki) and \
                            not explanationExists(3, explEki,
                                                  variants):
                          expl = explEki
                          print('Sõnaga, millega tehti päring: ', word)
                          meetod = 'unikaalne'
                          break
    return expl, meetod
    
# Meetod, mis eemaldab sünohulgast kõik liikmed, mis esinevad muudes sünohulkades.
def removeDuplicate(dictSynsets):
    if len(dictSynsets) <= 1:
        return dictSynsets
    unique = {}
    for n in dictSynsets:
        for m in dictSynsets:
            if n != m:
                newKey = set(n) - set(m)
                ## Jätan alles ainult need sõnad, mis esinevad antud seletuse puhul ning mitte kuskil mujal.
                newKey = onlyUniqueWords(newKey)
                unique[tuple(newKey)] = dictSynsets[n]
    return unique

# Teeb järjendi unikaalsetest sõnadest
def onlyUniqueWords(words):
    new = []
    for word in words:
        results = getWord(3, word)
        if len(results) == 1:
            new.append(word)
    return new


def makeDict(results):
    dictSynsets = {}
    for result in results:
        synset = processSynset(result)
        explanation = result['primary_definition']
        dictSynsets[tuple(synset)] = explanation
    return dictSynsets


In [29]:
# Meetod, mis arvutab välja sarnasuse Ekilexist saadava sünonüümide ja Sünohulga liikmete vahel.
def getSimilarity(variants):
    synSetSimilarity = {}
    wordIDs = {}
    for word in variants:
        ids, synonyms = getMeaningID(word)
        differenece = 0
        if ids != 'puudub':
            for i in range(len(synonyms)):
                syn = synonyms[i]
                synSetDiff = len(set(variants) & set(syn)) / float(len(set(variants) | set(syn)))
                if synSetDiff >= differenece:
                    differenece = synSetDiff
                    wordIDs[word] = ids[i]
                    synSetSimilarity[ids[i]] = synSetDiff
    return synSetSimilarity, wordIDs

In [30]:
# Numbrite käsitlemine. Neid ei arvestatud analüüsi, sest neid oli liiga vähe ning peamiselt panin seletuse ise kokku.
def numberExplanation(variants):
    expl = ''
    variants = [x for x in variants if not x[0].isupper() and x[-1] != '.']
    for number in variants:
        if number.isnumeric():
            expl = 'põhiarv ' + number
            break
        else:
            ids, syns = getMeaningID(number)
            expl = getWordMeaning(ids[0]) if ids != 'puudub' else ids
    return expl

# Meetod, et kätte saada sünohulga seletus
def synsetExplanation(lexid, variants):
    expl = 'puudub'
    meetod = 'puudub'
    synSetSimilarity, wordIDs = getSimilarity(variants) # Leian, kui palju on ühtivaid sünonüüme Ekilexi seletuse juures ja EstWNi sünohulga vahel.
    print('Sarnasused: ', synSetSimilarity)
    maxValue = max(synSetSimilarity.values()) if len(synSetSimilarity) != 0 else 'puudub' # Saan kätte seletuse ID, millel on kõige rohkem ühiseid sünonüüme sünohulgaga
    # Kui sarnasusi pole, siis kasutan unikaalse sünohulga liikme meetodit
    if maxValue == 0 or maxValue == 'puudub':
        results = getWord(lexid, variants[0])
        dictSynsets = makeDict(results)
        unique = removeDuplicate(dictSynsets) # Eemaldan kõik sünohulga liikmed, mis esinevad teistes sünohulkades.
        print('Algne:', dictSynsets)
        print('Unique: ', unique)
        expl, meetod = uniqueExpl(expl, unique, variants, meetod) # Kasutan unikaalse sünohulga liikme meetodit.
        # Kui unikaalse sünohulga liikme põhjal seletust ei leidnud 
        # kontrollin, kas tegemist on mine-tegijanimedest koosneva sünohulgaga
        if expl == 'puudub' and checkForm(variants, 'mine', 4):
          expl = correctForm(unique, variants, expl, 'mine')
          meetod = '-mine vorm' if expl != 'puudub' else 'puudub'
        # Kui unikaalse sünohulga liikme põhjal seletust ei leidnud 
        # kontrollin, kas tegemist on ja-tegijanimedest koosneva sünohulgaga
        elif expl == 'puudub' and checkForm(variants, 'ja', 2):
          expl = correctForm(unique, variants, expl, 'ja')
          meetod  = '-ja vorm' if expl != 'puudub' else 'puudub'
    # Sünonüümide sarnasuse põhjal seletuse leidmine.
    else:
        listMaxIDs = [k for k, v in synSetSimilarity.items() if v == maxValue]
        print('Maksmiaalsed võtmed: ', listMaxIDs)
        for wordID in listMaxIDs:
            explEki = getWordMeaning(wordID)
            if explEki != 'puudub' and explEki is not None:
              if len(explEki.split(' ')) > 1 and \
                    not synonymInExplanation(variants, explEki) and \
                    not explanationExists(3, explEki, variants):
                  expl = explEki
                  meetod = 'sarnasus'
                  break
    return expl, meetod

In [32]:
# 0ffset näitab, mitmendast alustab ja limit on mitu korraga annab. Default on 20.
offset = 0  
limit = 0       

## Muutuja, selleks, et vaadata mitmenda sõnaga mingi probleem tekib
tsükliteArv = 0

# Excelisse kirjutamine
wb = openpyxl.load_workbook('Tulemused(Koik).xlsx') # Vastavas GD kaustas, peab eksisteerima see fail, et koodi jooksutada
jrk = 0
sh1 = wb['Leht1']

while offset < 90000:
    results = getMissingMeanigns(3, offset)
    for result in results:
        # print(json.dumps(result, indent=4))
        tsükliteArv += 1
        # print('Tsükkel:', tsükliteArv)
        variants = processSynset(result)
        if variants and result['primary_definition'] == '':
            expl = ''
            jrk += 1
            translExpl = 'puudub'
            firstElem = variants[0]
            isNumber = len(firstElem) != 0 and firstElem[-1] == '.'
            if result['domain'] == 'Number' or result['domain'] == 'Number, Number' or isNumber:
                expl = numberExplanation(variants)
                meetod = 'number'
            else:
                print(variants)
                algneExpl, meetod = synsetExplanation(3, variants)
                cleanedExpl = re.sub('<.*?>', '', str(algneExpl))
                expl = 'puudub' if len(cleanedExpl.strip()) == 0 else cleanedExpl
                translExpl, engExpl = engExplanation(result['synset_externals']) 
                print('Ingliskeelne seletus olemas:', engExpl)
                if translExpl != 'puudub' and meetod == 'puudub':
                  meetod = 'tõlge' 
            print('{}. EKI seletus ({}): {} \nNeurotõlke (eng -> est) seletus: {}'.format(tsükliteArv, result['variants_str'], expl,translExpl))
            print('Jrk: {}; offset = {}; tsükliteArv = {}'.format(jrk, offset, tsükliteArv))
            print('=================================================')
            sh1.cell(row = jrk, column = 1, value = result['variants_str'])
            sh1.cell(row = jrk, column = 2, value = 'EKI seletus: {}'.format(expl))
            sh1.cell(row = jrk, column = 3, value = translExpl)
            sh1.cell(row = jrk, column = 4, value = meetod)
            wb.save('Tulemused(Koik).xlsx')
    offset += 20


FileNotFoundError: ignored